### Spatial Downsampling for Cross-Montage Comparison


In [ ]:
import numpy as np
import mne
#load standard montages and check if raw has 128 chs or more

bio128=mne.channels.make_standard_montage("biosemi128")
bio64=mne.channels.make_standard_montage("biosemi64")
#functions to determine the matching channels
def calculate_distance(arr1, arr2):
    # Assuming arr1 and arr2 are numpy arrays
    return np.linalg.norm(arr1 - arr2)

def find_close_channels(dict1, dict2, threshold):
    close_channel_pairs = []
    close_dict ={}
    for key1, value1 in dict1.items():
        # Check if the corresponding key exists in the second OrderedDict
        for key2 ,value2 in dict2.items():
            distance = calculate_distance(value1, value2)

            if distance < threshold:
                close_channel_pairs.append((key1, key2))
                close_dict[key1]=key2

    return close_channel_pairs , close_dict
#from montages get an orderDict with positions and check matching channels within thr
threshold = 1e-10
dict64 = bio64.get_positions()['ch_pos']
dict128 = bio128.get_positions()['ch_pos']

close_channel_pairs ,close_dict= find_close_channels(dict64, dict128, threshold)
print(f"{len(close_channel_pairs)} Pairs of keys closer than {threshold}:")
#channels to interpolate from 64 cap
cap64_chs_to_add = [ch for ch in bio64.ch_names if ch not in close_dict.keys()]
#channels to remove after interpolation
cap158_chs_to_remove = [ch for ch in bio128.ch_names if ch not in close_dict.values()]
#create custom montage
dict158 = dict128
#add channels from 64 montage that don't match any of the 128 location
for new_chs in cap64_chs_to_add:
    dict158[new_chs] = dict64[new_chs]
#these are the fiducials for both biosemi montages
nas = bio64.get_positions()['nasion']
lpa = bio64.get_positions()['lpa']
rpa = bio64.get_positions()['rpa']

custom_montage = mne.channels.make_dig_montage(ch_pos=dict158,coord_frame='head',nasion=nas,lpa=lpa,rpa=rpa)
custom_montage.plot()

#add new channels
sfreq = raw.info['sfreq']
n_samples = raw.n_times

# Create new channels with all zero values (30 is now hardcoded but it can be easyly change to be expected nchan-matching_chs_number)
new_channel_data = np.zeros((30, n_samples))

# Create an info structure for the new channels
new_channel_info = mne.create_info(ch_names=cap64_chs_to_add, sfreq=sfreq, ch_types=['eeg']*30)

# Create a RawArray object with the new channel data
new_channel_raw = mne.io.RawArray(new_channel_data, new_channel_info)

# Add the new channel to the Raw object
raw.add_channels([new_channel_raw], force_update_info=True)
tointer = raw.copy().set_montage(custom_montage, on_missing='ignore')
# check if fail tointer.info['dig']
tointer.info['bads'] = cap64_chs_to_add
#[eeg.info.ch_names.append(bad) for bad in cap64_chs_to_add]
tointer = tointer.pick_channels(custom_montage.ch_names)
raw_interpolated = tointer.interpolate_bads()
#remove unwanted channels
raw_interpolated.drop_channels(cap158_chs_to_remove)
#change names of the remainig channels
dict_close = {key: val for val , key in close_dict.items()}
mne.rename_channels(raw_interpolated.info, dict_close)
#reorder channels to the biosemi original order
raw_interpolated = raw_interpolated.reorder_channels(bio64.ch_names)